In [ ]:
!pip install pygame

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.7/13.7 MB 44.9 MB/s eta 0:00:00


In [4]:
import pygame
import random
import math

In [5]:
# Set up the display
WINDOW_WIDTH = 800 # Square dimension
WINDOW = pygame.display.set_mode((WINDOW_WIDTH,WINDOW_WIDTH))
pygame.display.set_caption("Maze solver")

# Colors for path finder visualization
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
PURPLE = (128, 0, 128)
ORANGE= (255, 165, 0)
GREY = (128, 128, 128)

In [6]:
def norm2(node1,node2):
  x1,y1 = node1
  x2,y2 = node2
  # Computes pitagorean distance among two nodes
  return ((x2-x1)**2+(y2-y1)**2)**0.5

In [13]:
def get_clicked_pos(pos, rows):
        # Returns the row and column in the pygame window corresponing to the clicked position

        gap = WINDOW_WIDTH//rows
        y,x = pos
        row = y//gap
        col = x//gap
        return (row,col)

In [43]:
class Node():
    # White nodes are free
    # Black nodes are obstacles
    # Blue node is start
    # Orange node is goal
    # Red nodes are valid nodes
    # Purple nodes are part of final path

    def __init__(self, row, col, radius):
        self.row = row
        self.col = col
        # Variables to draw the circles inside the display
        self.radius = radius//2
        self.x = row*radius
        self.y = col*radius
        # Node color not defined
        self.color = None
    
    def get_pos(self):
      # Outputs coordinates as a tuple
      return (self.row,self.col)

    def is_free(self):
        return self.color == WHITE
    
    def is_obstacle(self):
        return self.color == BLACK

    def is_start(self):
      return self.color == BLUE

    def is_goal(self):
        return self.color == ORANGE

    def set_free(self):
        self.color = WHITE
    
    def set_obstacle(self):
        self.color = BLACK

    def set_node(self):
        self.color = RED

    def set_start(self):
        self.color = BLUE

    def set_goal(self):
        self.color = ORANGE

    def set_path(self):
        self.color = PURPLE
    
    def draw_node(self):
      # Draws the node in the display
      pygame.draw.circle(WINDOW, self.color, (self.x,self.y), self.radius)
      pygame.display.update()

In [70]:
class Maze():
  def __init__(self, ROWS, COLUMNS,N_OBSTACLES,step_size):
    self.map = []
    self.rows = ROWS
    self.cols = COLUMNS
    self.start = None
    self.end = None
    self.step = step_size
    self.goal_found = False
    self.obsList = []
    self.AlgTable = {}
    rad = WINDOW_WIDTH // ROWS

    # Creates an empty map
    for row in range(ROWS):
      rowList = []
      for col in range(COLUMNS):
        node = Node(row, col, rad)
        node.set_free()
        rowList.append(node) # Append white node
      self.map.append(rowList)

      WINDOW.fill(WHITE)
      pygame.display.update()

  def set_startNode(self,node):
    # Sets the start node and draws it
    self.start = node
    node.set_start()
    node.draw_node()
    pygame.display.update()

  def set_endNode(self,node):
    # Sets the goal node and draws it
    self.end = node
    node.set_goal()
    node.draw_node()
    pygame.display.update()
  
  def reset_obsList(self):
    self.obsList = []

  def add_obstacle(self, first_position, second_position):
    x1, y1 = first_position
    x2, y2 = second_position
    # Retrieve obstacle width and height
    WIDTH = abs(x2-x1)
    HEIGHT = abs(y2-y1)
    print(WIDTH, HEIGHT)
    # For representation purposes, the left top corner coordinates must be retrieved
    x_start = min(x1,x2)
    y_start = min(y1,y2)
    # Draw the rectangle
    if WIDTH != 0 and HEIGHT!= 0:
      pygame.draw.rect(WINDOW, BLACK, (self.map[x_start][y_start].x, self.map[x_start][y_start].y, WIDTH*WINDOW_WIDTH//self.rows, HEIGHT*WINDOW_WIDTH//self.rows))
      pygame.display.update()
      # Set map nodes to obstacles
      for row in range(y_start,y_start + HEIGHT):
        for col in range(x_start, x_start + WIDTH):
          self.map[row][col].set_obstacle()
    
  def find_nearest(self,rand_node):
    # Given a randomly generated node (as a tuple with its coordinates (x,y)), finds its closest node and corresponding distance
    min_dis = (self.rows**2+self.cols**2)**0.5 # Initialized at maximum possible given grid dimension
    for node in self.AlgTable.keys():
      if dis := norm2(rand_node, node) < min_dis:
        min_dis = dis
        nearest_node = node

    return nearest_node, min_dis  

  def obstacles_on_the_way(self, rand_node, nearest_node, dis):
    # Checks if there are obstacles between two given nodes coordinates
    x_rand, y_rand = rand_node
    x_node, y_node = nearest_node

    for i in range(1,100):
      step = i/100
      x_new = x_node*step + x_rand*(1-step)
      y_new = y_node*step + y_rand*(1-step)
      if self.map[x_new][y_new].is_obstacle():
        return True

  def add_new_node(self, rand_node, nearest_node, dis):
    # Given the randomly generated node and its closest node and its distance, the new node to be added position is computed

    x_rand, y_rand = rand_node
    x_node, y_node = nearest_node

    # If the nodes are closer than the step size, directly use the randomly generated line
    if dis < self.step_size:
      x_new, y_new = rand_node
    # Otherwise, place the node at a distance step size from nearest node along the direction connecting the randomly generated node and its closest
    else:
      th = math.atan2(y_rand - y_node, x_rand - x_node)
      x_new = int(x_node + self.step_size*math.cos(th))
      y_new = int(y_node + self.step_size*math.sin(th))
    
    # If the new node is close enough to the goal node, directly use the goal node
    if norm2((x_new, y_new), self.end.get_pos()):
      x_new, y_new = self.end.get_pos()
      self.goal_found = True

    # Add the newly found node to the table, draw it and connect it to its closest node
    self.AlgTable[(x_new,y_new)] = nearest_node
    self.map[x_new][y_new].set_node()
    self.map[x_new][y_new].draw_node()
    pygame.draw.line(RED, (x_new,y_new), nearest_node)
    pygame.display.update()

  def retrieve_path(self):
    current = self.end.get_pos()
    while current != self.start.get_pos():
      self.map[current[0]][current[1]].set_path()
      self.map[current[0]][current[1]].draw_node()
      pygame.draw.line(WINDOW, PURPLE, current, self.AlgTable[current])
      current = self.AlgTable[current]
      pygame.display.update()

  def RRT_path_finder(self):
    # Implements RRT algorithm
    # Only active node at the start is the start node
    self.AlgTable[self.start.get_pos()] = None

    while not self.goal_found:
      # Initialize a random node
      x_rand, y_rand = random.randint(0,self.cols), random.randint(0,self.rows)
      # Verify it is not an obstacle
      if self.map[x_rand][y_rand].is_obstacle():
        continue
      nearest_node, distance = self.find_nearest((x_rand,y_rand))
      if self.obstacles_on_the_way((x_rand,y_rand), nearest_node, distance):
        continue
      self.add_new_node((x_rand,y_rand), nearest_node, distance)
      if self.goal_found:
        self.retrieve_path()

In [71]:
def main():
    DIM = 50 # Maze dimension (square)
    # Create a maze instance
    myMaze = Maze(DIM, DIM, 3, 5)

    started = False
    run = True

    while run:
        for event in pygame.event.get():  # Detects actions while display ON
            if event.type == pygame.QUIT: 
                run = False

            if pygame.mouse.get_pressed()[0]: # If left mouse button is pressed
                pos = pygame.mouse.get_pos()
                row,col = get_clicked_pos(pos, DIM) # Retrieve row, col in maze
                spot = myMaze.map[row][col] # Get the node in the clicked position
                # In case the start was not defined, make it
                if not myMaze.start and not spot.is_goal():
                    myMaze.set_startNode(spot)
                
                # In case the end was not defined, make it
                elif not myMaze.end and not spot.is_start():
                    
                    myMaze.set_endNode(spot)

                elif myMaze.start and myMaze.end:
                    myMaze.obsList.append((row,col))
                    print(myMaze.obsList)
                    spot.set_obstacle()
                    spot.draw_node
                    if len(myMaze.obsList) == 2:
                        myMaze.add_obstacle(myMaze.obsList[0], myMaze.obsList[1])
                        myMaze.reset_obsList()

            elif pygame.mouse.get_pressed()[2]: # If right mouse button is pressed
                pos = pygame.mouse.get_pos()
                row,col = get_clicked_pos(pos, DIM) # Retrieve row, col in maze
                spot = myMaze.map[row][col] # Get the node in the clicked position
                spot_pos = spot.get_pos()
                # Reset the spot unless it is an obstacle
                if myMaze.start:
                    print("There is a start")
                    if norm2(spot.get_pos(), myMaze.start.get_pos()) < myMaze.start.radius:
                        myMaze.start.set_free()
                        print(myMaze.start.color)
                        print(myMaze.start.row, myMaze.start.col)
                        myMaze.map[myMaze.start.row][myMaze.start.col].draw_node()
                        myMaze.start = None
                        pygame.display.update()
                        
                if myMaze.end:
                    print("There is an end")
                    if norm2(spot.get_pos(), myMaze.end.get_pos()) < myMaze.end.radius:
                        myMaze.end.set_free()
                        myMaze.map[myMaze.end.row][myMaze.end.col].draw_node()
                        myMaze.end = None
                        pygame.display.update()
            
            if event.type == pygame.KEYDOWN:
                # If start and end are defined and blank space is pressed, start the A* algorithm
                if event.key == pygame.K_SPACE and not started and myMaze.start and myMaze.end:
                    myMaze.RRT_path_finder()

                if event.key == pygame.K_r and myMaze.start and myMaze.end: # If R is pressed on keyboard, reset everything
                    myMaze = Maze(DIM, DIM)

    pygame.quit()    

if __name__ == "__main__":
    main()

[(34, 31)]
[(34, 31), (12, 13)]
22 18
[(33, 44)]
[(33, 44), (33, 44)]
0 0
[(42, 40)]
[(42, 40), (42, 40)]
0 0
[(42, 40)]
[(42, 40), (42, 40)]
0 0
[(36, 2)]
[(36, 2), (42, 12)]
6 10
[(42, 12)]
[(42, 12), (41, 36)]
1 24
[(41, 36)]
[(41, 36), (32, 41)]
9 5


TypeError: list indices must be integers or slices, not float